In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.absolute()))
import config
import time
from context import Context
from multiversx_sdk.core import Address, AddressComputer
from multiversx_sdk.core import SmartContractTransactionsFactory
from multiversx_sdk.core import TransactionsFactoryConfig
from multiversx_sdk import ProxyNetworkProvider
from multiversx_sdk.core import TokenPayment, TokenComputer
from utils.utils_chain import WrapperAddress as Address, Account
from utils.utils_tx import endpoint_call
from contracts.price_discovery_contract import PriceDiscoveryContract
from multiversx_sdk.abi import Abi

context = Context()


proxy = ProxyNetworkProvider(config.devnet)
network_config = proxy.get_network_config()
net = "devnet" if "devnet" in proxy.url else "testnet" if "testnet" in proxy.url else "mainnet"
abi = Abi.load(Path(
    "/home/multiversx/Documents/mock_fwk/wasm/price-discovery/price-discovery.abi.json"
))
factory = SmartContractTransactionsFactory(TransactionsFactoryConfig(network_config.chain_id), abi)

factory_config = TransactionsFactoryConfig(network_config.chain_id)
# transactions_factory = SmartContractTransactionsFactory(factory_config, abi)



2025-02-17 09:47:04,173 - WARNING - Structure definition for stakings_boosted not found in /home/multiversx/Documents/mock_fwk/dex_deploy/configs-mainnet/deploy_structure.json


loaded 1 accounts from 1 PEM files.


2025-02-17 09:47:04,467 - DEBUG - Account.sync_nonce() done: 0
2025-02-17 09:47:04,623 - DEBUG - Account.sync_nonce() done: 0
2025-02-17 09:47:04,624 - INFO - Loaded 10 tokens.
2025-02-17 09:47:04,805 - DEBUG - Account.sync_nonce() done: 0
2025-02-17 09:47:04,806 - INFO - Loaded 0 stakings_boosted.
2025-02-17 09:47:04,897 - DEBUG - Account.sync_nonce() done: 0
2025-02-17 09:47:04,898 - INFO - Deployed contracts below:
2025-02-17 09:47:04,899 - INFO - stakings_boosted:


# SETUP

In [2]:
launched_token_id = "TKNZPWA-f0763d"
accepted_token_id = "TKNZPWA-a8b1bc"

user_min_deposit: int = 900000
admin = "erd1zpwadzzylule6njjtrrhf0hcysak23a76983tq6j7hddn66edr6q9qjytp"

deployer = Account(pem_file=config.DEFAULT_OWNER) 
deployer.sync_nonce(context.network_provider.proxy)
deployer_shard = AddressComputer().get_shard_of_address(deployer.address)



2025-02-17 09:47:06,947 - DEBUG - Account.sync_nonce() done: 5652


In [18]:
from utils import blocktime


start_time: int = "25/02/10 11:25"
user_deposit_withdraw_time: int = "25/02/17 10:30"
owner_deposit_withdraw_time: int = "25/02/17 10:40"

print(f"Start round calculation below:")
_, start_round , start_timestamp = blocktime.main([f"--net={net}", f"--shard={deployer_shard}", f"--date={start_time}"])
print(start_timestamp)

print("===========================================")
print(f"User deposit end round calculation below:")
_, start_round, user_deposit_timestamp = blocktime.main([f"--net={net}", f"--shard={deployer_shard}", f"--date={user_deposit_withdraw_time}"])
print(user_deposit_timestamp)
user_deposit_time = user_deposit_timestamp-start_timestamp
print("Seconds untill user deposit time ends:", user_deposit_timestamp-start_timestamp)

print("===========================================")
print(f"Owner deposit end round calculation below:")
_, start_round, owner_deposit_timestamp = blocktime.main([f"--net={net}", f"--shard={deployer_shard}", f"--date={owner_deposit_withdraw_time}"])
print(owner_deposit_timestamp)
owner_deposit_time = owner_deposit_timestamp-user_deposit_timestamp
print("Seconds untill owner deposit time ends:", owner_deposit_time)



Start round calculation below:
Calculating from block: 7519740 round: 7590844 shard: 0
Current date: 2025-02-14 16:57:44 (timestamp of last produced block)
Target date: 2025-02-14 17:25:00
Target time: 1739546700
1739546700
User deposit end round calculation below:
Calculating from block: 7519740 round: 7590844 shard: 0
Current date: 2025-02-14 16:57:44 (timestamp of last produced block)
Target date: 2025-02-14 17:30:00
Target time: 1739547000
1739547000
Seconds untill user deposit time ends: 300
Owner deposit end round calculation below:
Calculating from block: 7519740 round: 7590844 shard: 0
Current date: 2025-02-14 16:57:44 (timestamp of last produced block)
Target date: 2025-02-14 17:40:00
Target time: 1739547600
1739547600
Seconds untill owner deposit time ends: 600


# DEPLOY

In [19]:
contract = PriceDiscoveryContract(launched_token_id, accepted_token_id,"",start_timestamp, user_deposit_timestamp, owner_deposit_timestamp,user_min_deposit,admin)
args = [
        launched_token_id, 
        accepted_token_id,
        18,
        start_timestamp,
        user_deposit_time,
        owner_deposit_time,
        user_min_deposit,
        deployer.address
        ]
# provider = ProxyNetworkProvider(config.DEFAULT_PROXY) 
# deployer.address = Address.from_bech32("erd1wwx5zhmx9mag9k8zuajj8c9zce6mzerfr3m9ck6l7c00dwkq2a2ssyfnzk")
# tx = contract.contract_deploy(deployer, context.network_provider.proxy, config.PRICE_DISCOVERY_BYTECODE_PATH, [])
# tx = "1b3dacf1507ef888a8f1bd86e2749d23ec7d3b83106f7cb608a961bb465116ba"
# tx_status = context.network_provider.check_simple_tx_status(tx)

tx = factory.create_transaction_for_deploy(deployer.address, config.PRICE_DISCOVERY_BYTECODE_PATH, 150000000, arguments=args)
tx.nonce = deployer.nonce
tx.signature = deployer.sign_transaction(tx)

hash = proxy.send_transaction(tx)
deployer.nonce += 1
time.sleep(6)
if proxy.get_transaction_status(hash).is_failed():
    raise Exception(f"Transaction failed: {hash}")

receipt = proxy.get_transaction(hash)
contract_address = receipt.logs.find_first_or_none_event("SCDeploy").address

print(f"Contract deployed at address: {contract_address.to_bech32()}")

contract.address = contract_address.to_bech32()


Contract deployed at address: erd1qqqqqqqqqqqqqpgqhtn0fxs4wjf3zm4j09ye8wjrxwvx2wx6dr6qs6cny6


# SET CONTRACT DATA

In [21]:
contract = PriceDiscoveryContract(launched_token_id, accepted_token_id,"",start_timestamp, user_deposit_timestamp, owner_deposit_timestamp,user_min_deposit,admin)
contract.address = "erd1qqqqqqqqqqqqqpgqhtn0fxs4wjf3zm4j09ye8wjrxwvx2wx6dr6qs6cny6"
# contract.address = Address.new_from_bech32(contract.address)
contract.print_contract_info()

2025-02-14 16:59:08,919 - INFO - Deployed price discovery contract: erd1qqqqqqqqqqqqqpgqhtn0fxs4wjf3zm4j09ye8wjrxwvx2wx6dr6qs6cny6
2025-02-14 16:59:08,920 - INFO -  ├ Redeem token: 
2025-02-14 16:59:08,921 - INFO -  ├ Start time: 1739546700


ISSUE REDEEM TOKEN

In [22]:
redeem_token_name = "REDEEMT"

In [23]:


redeem_token = contract.issue_redeem_token(deployer, proxy, redeem_token_name)
print(redeem_token)

2025-02-14 16:59:30,821 - INFO - Issue price discovery redeem token
2025-02-14 16:59:30,823 - DEBUG - Calling issueRedeemToken at erd1qqqqqqqqqqqqqpgqhtn0fxs4wjf3zm4j09ye8wjrxwvx2wx6dr6qs6cny6
2025-02-14 16:59:30,994 - DEBUG - Contract call arguments: ['REDEEMT', 'REDEEMT', 18]
The transaction builders are deprecated and will be removed in a later release. Use the transaction factories instead.
2025-02-14 16:59:31,168 - INFO - View this transaction in the MultiversX Devnet Explorer: https://devnet-explorer.multiversx.com/transactions/c054f4b10207722282391d4d5dc4a4b1eb05e1ff9db39f2f076b3d291a50d750


c054f4b10207722282391d4d5dc4a4b1eb05e1ff9db39f2f076b3d291a50d750


SET TRANSFER ROLE

In [25]:
from multiversx_sdk import Address, TransactionComputer

user = Account(pem_file="/home/multiversx/Documents/mock_fwk/wallet1.pem")
user.sync_nonce(proxy)
# contract_address=Address.new_from_bech32("erd1qqqqqqqqqqqqqpgqpwe5z60vqmnjluq7cqzdrs3gvsghsy09dr6qt8v32z")
# event = DepositPDLiquidityEvent(launched_token_id, amount)

set_transfer_role_farm_token_tx = factory.create_transaction_for_execute(
    sender=user.address,
    contract=Address.new_from_bech32(contract.address),
    function="setTransferRole",
    gas_limit=70000000,
)

set_transfer_role_farm_token_tx.nonce = user.nonce
set_transfer_role_farm_token_tx.signature = user.signer.sign(
    TransactionComputer().compute_bytes_for_signing(set_transfer_role_farm_token_tx)
)

result = proxy.send_transaction(set_transfer_role_farm_token_tx)

2025-02-14 17:00:47,077 - DEBUG - Account.sync_nonce() done: 5651


SET MIN LAUNCHED TOKENS

In [24]:
from multiversx_sdk import Address, TransactionComputer

user = Account(pem_file="/home/multiversx/Documents/mock_fwk/wallet1.pem")
user.sync_nonce(proxy)
amount = 10000 * 10 ** 18


set_min_launched_tx = factory.create_transaction_for_execute(
    sender=user.address,
    contract=Address.new_from_bech32(contract.address),
    function="setMinLaunchedTokens",
    gas_limit=70000000,
    arguments = [amount]
)

set_min_launched_tx.nonce = user.nonce
set_min_launched_tx.signature = user.signer.sign(
    TransactionComputer().compute_bytes_for_signing(set_min_launched_tx)
)

result = proxy.send_transaction(set_min_launched_tx)

2025-02-14 17:00:20,042 - DEBUG - Account.sync_nonce() done: 5650


CHANGE OWNER

In [ ]:
second_owner = Account(pem_file=config.DEFAULT_ADMIN)
second_owner.sync_nonce(proxy)

first_owner = Account(pem_file=config.DEFAULT_OWNER)
first_owner.sync_nonce(proxy)

user = Account(pem_file=config.DEFAULT_ACCOUNTS)
user.address = Address("erd1zpwadzzylule6njjtrrhf0hcysak23a76983tq6j7hddn66edr6q9qjytp")
user.sync_nonce(proxy)

In [ ]:
hash = endpoint_call(proxy, 50000000, second_owner, contract_address, 
                     "ChangeOwnerAddress", [first_owner.address])

# WHITELIST

In [ ]:
whitelisted_address = []
whitelisted_address.append("erd1my8mcw2fhhsvgtzah7ntr9sdm9zxlamul6uhksyfcvzagemwad7qprplwz")
whitelisted_address.append("erd1gvkklm20rk9vg0xnyq0aq3ae3cnle8qxa7eqcevnhfthe5gj9z4s293za5")

deployer.sync_nonce(proxy)
contract.add_user_to_whitelist(proxy, deployer, whitelisted_address)

# DEPOSIT

OWNER DEPOSIT

In [ ]:
from events.price_discovery_events import DepositPDLiquidityEvent

user = Account(pem_file="/home/multiversx/Documents/mock_fwk/wallet1.pem")
user.sync_nonce(proxy)
amount = 10000 * 10 ** 18
event = DepositPDLiquidityEvent(launched_token_id, amount)

contract.deposit_liquidity(context.network_provider.proxy, user, event)

USER DEPOSIT

In [ ]:
from events.price_discovery_events import DepositPDLiquidityEvent

user = Account(pem_file="/home/multiversx/Documents/mock_fwk/wallet3.pem")
user.sync_nonce(proxy)
amount = 100 * 10 ** 18
event = DepositPDLiquidityEvent(accepted_token_id, amount)

contract.user_deposit(context.network_provider.proxy, user, event)

USER WITHDRAW

In [ ]:
from events.price_discovery_events import WithdrawPDLiquidityEvent
# contract.address=Address.new_from_bech32("erd1qqqqqqqqqqqqqpgqpwe5z60vqmnjluq7cqzdrs3gvsghsy09dr6qt8v32z")

user = Account(pem_file="/home/multiversx/Documents/mock_fwk/wallet2.pem")
user.sync_nonce(proxy)
amount = 100 * 10 ** 18


event = WithdrawPDLiquidityEvent("REDEEMT-d05574",0, amount)

contract.user_withdraw(context.network_provider.proxy, user, event)

OWNER WITHDRAW

In [ ]:
from events.price_discovery_events import WithdrawPDLiquidityEvent
# contract.address=Address.new_from_bech32("erd1qqqqqqqqqqqqqpgqpwe5z60vqmnjluq7cqzdrs3gvsghsy09dr6qt8v32z")

user = Account(pem_file="/home/multiversx/Documents/mock_fwk/wallet2.pem")
user.sync_nonce(proxy)
amount = 100 * 10 ** 18


event = WithdrawPDLiquidityEvent("TKNZPWA-a8b1bc",0, amount)

contract.withdraw_liquidity(context.network_provider.proxy, user, event)

# REDEEM

In [ ]:
from events.price_discovery_events import RedeemPDLPTokensEvent
# contract.address=Address.new_from_bech32("erd1qqqqqqqqqqqqqpgq87q5q0zannvd7s2jeycw36j9trnyzskydr6qvu3p32")

user = Account(pem_file="/home/multiversx/Documents/mock_fwk/wallet3.pem")
user.sync_nonce(proxy)
amount = 100 * 10 ** 18


event = RedeemPDLPTokensEvent("REDEEMT-d05574",0, amount)

contract.redeem_liquidity_position(context.network_provider.proxy, user, event)

In [ ]:
from events.price_discovery_events import RedeemPDLPTokensEvent
# contract.address=Address.new_from_bech32("erd1qqqqqqqqqqqqqpgq87q5q0zannvd7s2jeycw36j9trnyzskydr6qvu3p32")

user = Account(pem_file="/home/multiversx/Documents/mock_fwk/wallet1.pem")
user.sync_nonce(proxy)
amount = 100 * 10 ** 18


event = RedeemPDLPTokensEvent(accepted_token_id,0, amount)

contract.owner_redeem(context.network_provider.proxy, user, event)

# SEND TOKENS

In [ ]:
from utils.utils_chain import Account, WrapperAddress
from utils.utils_tx import ESDTToken, multi_esdt_transfer

# sender = Account(pem_file=config.DEFAULT_ACCOUNTS)

# sender.address = WrapperAddress('erd1q3yux0lm7vyutdxyrq7ad762mavurxpuhtuxlwau06eqyr49yknqh6vkk2')
receiver = WrapperAddress('erd1my8mcw2fhhsvgtzah7ntr9sdm9zxlamul6uhksyfcvzagemwad7qprplwz')

tokens = [
    ESDTToken('TKNZPWA-a8b1bc', 0, 10000000000000000000000),
]

deployer.sync_nonce(context.network_provider.proxy)
multi_esdt_transfer(context.network_provider.proxy, 1000000, deployer, receiver, tokens)

# REFUND

In [6]:
from multiversx_sdk import Address

users = []
users.append(("erd1my8mcw2fhhsvgtzah7ntr9sdm9zxlamul6uhksyfcvzagemwad7qprplwz"))
users.append("erd1gvkklm20rk9vg0xnyq0aq3ae3cnle8qxa7eqcevnhfthe5gj9z4s293za5")

deployer.sync_nonce(proxy)
contract.refund_users(proxy, deployer, users)

2025-02-14 14:49:26,397 - DEBUG - Account.sync_nonce() done: 5647
2025-02-14 14:49:26,398 - INFO - Refund users
2025-02-14 14:49:26,399 - DEBUG - Account: erd1zpwadzzylule6njjtrrhf0hcysak23a76983tq6j7hddn66edr6q9qjytp


QUERIES

IS WHITELISTED

In [15]:
from multiversx_sdk import Address

args = []
args.append(Address.new_from_bech32("erd1my8mcw2fhhsvgtzah7ntr9sdm9zxlamul6uhksyfcvzagemwad7qprplwz"))
contract.contract_query(proxy,deployer,"isUserWhitelisted",args)

[True]

GET CURRENT PRICE

In [10]:
args = []
contract.contract_query(proxy,deployer,"getCurrentPrice",args)

[40000000000000000]

GET CURRENT PHASE

In [9]:
args = []
contract.contract_query(proxy,deployer,"getCurrentPhase",args)

[namespace(__discriminant__=3)]

GET USER DEPOSIT LIMIT

In [8]:
args = []
args.append(Address.new_from_bech32("erd1my8mcw2fhhsvgtzah7ntr9sdm9zxlamul6uhksyfcvzagemwad7qprplwz"))
contract.contract_query(proxy,deployer,"getUserDepositLimit",args)

[0]